In [144]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


sns.set_style('darkgrid')


def class_boundary(female_prob, male_prob, female_act, male_act,title,savepath):
    
    x = np.arange(1, len(female_prob)+1)
    sns.lineplot(x, female_prob)
    sns.lineplot(x, male_prob)
    sns.lineplot(x, female_act)
    sns.lineplot(x, male_act)
    
    plt.legend(['female_prob', 'male_prob','female_act','male_act'],prop={'size':14})
    plt.title(title)
    plt.xlabel('Image number')
    plt.ylabel('Probability')

    plt.savefig(savepath,dpi= 300)
    plt.show()
    
    
def diff_image(female_prob, male_prob, female_act, male_act,savepath):
     x = np.arange(1, len(female_prob)+1)
     
     fig, ax1 = plt.subplots()
     
     ax1.set_xlabel('Image number')
     ax1.set_ylabel('Probability')
     ax1.plot(x,female_prob,color='darksalmon')
     ax1.plot(x,male_prob,color='cornflowerblue')
     ax1.legend(['female probability', ' male   probability'], bbox_to_anchor=(0.55,1.2),loc="upper left")
     
     ax2 = ax1.twinx()
     ax2.set_ylabel('Activation')
     ax2.plot(x,female_act,color='red')
     ax2.plot(x,male_act,color='blue')
     ax2.legend(['female activation',' male   activation'],bbox_to_anchor=(0.45,1.2),loc="upper right")
     
     fig.tight_layout()
     plt.savefig(savepath,dpi= 300)
     plt.show()

In [48]:
import pandas as pd

data = pd.read_csv(r'D:\cnnface\gender_analysis\supplementray_analysis\morph_face_output/morphface_result.csv')
female_prob = data['female_probability']  
male_prob = data['male_probability']
female_act = data['female_activation']  
male_act = data['male_activation']


In [145]:
#title = 'The classification activation and probability of 500 morph faces'
savepath = 'D:\cnnface\gender_analysis\supplementray_analysis\plot/' \
           'The classification activation and probability of 500 morph faces.png'

diff_image(female_prob, male_prob, female_act, male_act,savepath)

In [ ]:
# 
import numpy as np
import pandas as pd

label = np.load(r'D:\cnnface\gender_analysis\noise_stimulus\label/gender_label_20000.npy')
label_prob = np.load(r'D:\cnnface\gender_analysis\noise_stimulus\label/gender_label_pro_20000.npy')
dnn_act = np.load(r'D:\cnnface\gender_analysis\noise_stimulus\label/gender_label_act_20000.npy')

noiseface_result = pd.DataFrame({'label':label,
                                'female_probability':label_prob[:,0],
                                'male_probability':label_prob[:,1],
                                'female_activation':dnn_act[:,0],
                                'male_activation':dnn_act[:,1]})
noiseface_result.to_csv(r'D:\cnnface\gender_analysis\noise_stimulus\label/noiseface_result.csv')

In [11]:
import numpy as np
import matplotlib.pyplot as plt
from cnnface.analysis.generate_ci import cal_paramci,generateCI
# calculate the different frequency scale cis for human 
param_ci = np.load(r'D:\cnnface\gender_analysis\CI_analysis/param_ci_cnn.npy')
scale = [2,4,8,16,32]
cis = generateCI(param_ci,level=scale)

for ci, s in zip(cis,scale):
    plt.imshow(ci,cmap='jet')
    plt.axis('off')
    plt.title(s)
    plt.savefig(r'D:\cnnface\gender_analysis\CI_analysis\CIs\cis_img/{}.png'.format(s),dpi=300)
    plt.show()
np.save(r'D:\cnnface\gender_analysis\CI_analysis\CIs/cis_vgg.npy',cis)

In [16]:
import numpy as np
from cnnface.stimuli.image_manipulate import img_similarity
# calculate the correlation between cis of human and cis of alexnet

cis_alexnet = np.load(r'D:\cnnface\gender_analysis\supplementray_analysis\ci_result/cis_alexnet.npy')
cis_vggface = np.load(r'D:\cnnface\gender_analysis\human_result\CIs/cis_human.npy')

imgs_similarity = []
for ci_alex,ci_vggface in zip(cis_alexnet,cis_vggface):
    similarity = img_similarity(ci_alex,ci_vggface,method='pearsonr')
    imgs_similarity.append(similarity)

In [17]:
for i in imgs_similarity:
    print(i)
np.save(r'D:\cnnface\gender_analysis\supplementray_analysis\cis_correlation/correlationBetweenHuman2Alexnet.npy',imgs_similarity)

In [19]:
import numpy as np
from cnnface.stimuli.image_manipulate import img_similarity
ci_vgg = np.load(r'D:\cnnface\gender_analysis\CI_analysis/ci_cnn.npy')
ci_alexnet = np.load(r'D:\cnnface\gender_analysis\supplementray_analysis\ci_result/ci_alexnet.npy')
similarity = img_similarity(ci_alexnet,ci_vgg,method='pearsonr')

print(similarity)

In [22]:
plt.imshow(ci_alexnet,'jet')


In [8]:
# rebuld the ci-face from alexnet_gender ci
import numpy as np
from PIL import Image
from cnnface.analysis.generate_ci import cal_paramci,generateCI,recon_face

In [ ]:
# rebuild the category face
baseface = Image.open(r'D:\cnnface\gender_analysis\face_template\gray/baseface.jpg')
ci = np.load(r'D:\cnnface\gender_analysis\supplementray_analysis\ci_result/ci_alexnet.npy')

img_add, img_sub = recon_face(baseface,ci,50)
img_add.save(r'D:\cnnface\gender_analysis\supplementray_analysis\reconstruct_face/bf_add.jpg')
img_sub.save(r'D:\cnnface\gender_analysis\supplementray_analysis\reconstruct_face/bf_sub.jpg')

In [31]:
# rebuild the category face in each scale
baseface = Image.open(r'D:\cnnface\gender_analysis\face_template\gray/baseface.jpg')
cis = np.load(r'D:\cnnface\gender_analysis\supplementray_analysis\ci_result/cis_alexnet.npy')

level = (2, 4, 8, 16, 32)

for i, l in enumerate(level):
    print(cis[i, :, :].max())
    print(cis[i, :, :].min())
    img_add, img_sub = recon_face(baseface, cis[i, :, :],100)
    img_add.save(r'D:\cnnface\gender_analysis\supplementray_analysis\reconstruct_face\scales_68/bf_add100_%04d.jpg' % l)
    img_sub.save(r'D:\cnnface\gender_analysis\supplementray_analysis\reconstruct_face\scales_68/bf_sub100_%04d.jpg' % l)


In [ ]:
# get activation of different scale cis.


In [30]:
# plot activation 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# load the data of activation when cnn categorizing reconstructed face
resultAlexFace = pd.read_csv(r'D:\cnnface\gender_analysis\supplementray_analysis\reconstruct_face/alex_gender_result.csv')
# the first figure : bar plot x = scale, y = human_male_act, and cnn_male_act
actAlexMale = resultAlexFace['male_activation'][5:]
actAlexFemale = resultAlexFace['female_activation'][:5]

#act = actAlexFemale
act = actAlexFemale
Scale = [2,4,8,16,32] 

sns.set_style('darkgrid')
sns.barplot(x=Scale,y=act)
plt.show()

In [44]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

noiseface_data = pd.read_csv(r'D:\cnnface\gender_analysis\noise_stimulus\label/noiseface_result.csv')

male_activation = np.array(noiseface_data['male_activation'])
male_baseline = np.mean(male_activation)

female_activation = np.array(noiseface_data['female_activation'])
female_baseline = np.mean(female_activation)

image = list(range(20000))*4
female_activation  = list(female_activation)
female_activation  = [-i for i in female_activation]
act =   [female_baseline]*20000 + [0]*20000 + list(male_activation) +female_activation 
hue =  ['act_baseline']*20000 + ['origin_baseline']*20000 + ['male_activation']*20000 +['female_activation']*20000 
pl = sns.lineplot(x=image,y=act,hue=hue)
pl.text(10, 5, "act_baseline={}".format(round(female_baseline,2)), size='medium', color='black')
plt.savefig(r'D:\cnnface\gender_analysis\supplementray_analysis\difference_act_prob_method/female_male_activation.jpg',dpi=300)
plt.show()

In [42]:
len(hue)

In [31]:
male_activation_wave = male_activation - male_baseline
male_label = np.array([1 if a > 0 else 0 for a in male_activation_wave])

female_activation_wave = female_activation - female_baseline
female_label = np.array([0 if a > 0 else 1 for a in female_activation_wave])

In [32]:
overlap = [1 if m ==f else 0 for m,f in zip(male_label,female_label)]

In [36]:
print(sum(overlap)/20000)

